In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy.signal
import scipy.fft as sfft
import matplotlib.pylab as plt
from ipywidgets import interact, FloatSlider, IntSlider, SelectionSlider
from IPython.display import YouTubeVideo, Audio

# Actividad 1: Diseño de un filtro pasabajo

Considere los siguientes datos. Usemos `scipy.signal.firwin` para diseñar un filtro FIR pasa bajo con el método de la ventana que nos permita recuperar la señal intrínseca

Visualicemos la respuesta al impulso del filtro, la respuesta en frecuencia del filtro y el espectro de la señal resultante

¿Cómo cambia el resultado en función de la frecuencia de corte y $L$?

In [ ]:
Fs = 1
n = np.arange(0, 100, step=1/Fs); 
np.random.seed(0);
C = 5*np.exp(-0.5*(n[:, np.newaxis] - n[:, np.newaxis].T)**2/10**2)
x_clean = np.random.multivariate_normal(np.zeros_like(n), C)
x_clean -= np.mean(x_clean)
x = x_clean + 2*np.random.randn(len(n))

f = sfft.rfftfreq(n=len(n), d=1/Fs)
X = sfft.rfft(x)

fig, ax = plt.subplots(1, 2, figsize=(8, 3), tight_layout=True)
ax[0].plot(n, x_clean, label='señal intrínseca')
ax[0].scatter(n, x, c='k', s=5, label='datos observados')
ax[0].legend()
ax[1].plot(f, np.absolute(sfft.rfft(x_clean)), label='Espectro intrínsco')
ax[1].plot(f, np.absolute(X), c='k', label='Espectro observado')
ax[1].legend()

# Actividad 2: Filtrando una señal de audio

Considere la siguiente señal

In [ ]:
import librosa
data, sample_rate = librosa.load("../data/nomekop.ogg")

fig, ax = plt.subplots(1, figsize=(8, 3), tight_layout=True)

def plot_spectrogram(ax, data):
    freq, ttime, Sxx = scipy.signal.spectrogram(data, fs=sample_rate, 
                                                window=('kaiser', 6.), 
                                                nperseg=512, noverlap=512//1.5, 
                                                mode='magnitude')

    ax.pcolormesh(ttime, freq, 20*np.log10(Sxx+1e-4), shading='auto', cmap=plt.cm.magma); 
    ax.set_ylim([0.0, 1e+4]);
    ax.set_xlabel('Tiempo [s]')
    ax.set_ylabel('Frecuencia [Hz]')
    
plot_spectrogram(ax, data)

Audio(data, rate=int(sample_rate), normalize=False)

Diseñemos filtros usando `scipy.signal.firwin` 

- ¿Puedes crear un filtro borre a Blastoise y preserve a Squirtle?
- ¿Puedes crear un filtro borre a Squirtle y preserve a Blastoise?
- ¿Cómo cambia el resultado con $L$ y el tipo de ventana?

In [ ]:
h = scipy.signal.firwin(numtaps=1000+1, cutoff=[200, 500], 
                        pass_zero=False, window='hann',
                        fs=sample_rate)

data_filt = scipy.signal.convolve(data, h)


In [ ]:
freq_response, response = scipy.signal.freqz(h, fs=sample_rate)
fig = plt.figure(figsize=(7, 4), tight_layout=True)
ax1 = plt.subplot2grid((2, 2 ), (0, 0)); ax1.set_title("Respuesta al impulso")
ax2 = plt.subplot2grid((2, 2 ), (0, 1)); ax2.set_title("Respuesta en frecuencia")
ax3 = plt.subplot2grid((2, 2 ), (1, 0), colspan=2, rowspan=1)
ax1.plot(h); 
ax2.semilogy(freq_response, np.abs(response));
plot_spectrogram(ax3, data_filt)
Audio(data_filt, rate=int(sample_rate), normalize=False)